### Modeling

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [ ]:
X_train_resampled_controlled_df = pd.read_csv('X_train_preprocessed.csv')
y_train_resampled_controlled_df = pd.read_csv('y_train_preprocessed.csv')

X_test_transformed_df = pd.read_csv('X_test_transformed.csv')
y_test_df = pd.read_csv('y_test.csv')

The first model I am going to use is Logistic Regression.

In [ ]:
lr_model = LogisticRegression()

#convert to 1D array
y_train_flat = y_train_resampled_controlled_df.values.ravel()

lr_model.fit(X_train_resampled_controlled_df, y_train_flat)

In [ ]:
y_pred_lr = lr_model.predict(X_test_transformed_df)

#convert to 1D array
y_test_flat = y_test_df.values.ravel()

accuracy = accuracy_score(y_test_flat, y_pred_lr)
print("Accuracy:", accuracy)

conf_matrix = confusion_matrix(y_test_flat, y_pred_lr)
print("Confusion Matrix:")
print(conf_matrix)

class_report = classification_report(y_test_flat, y_pred_lr)
print("Classification Report:")
print(class_report)

Which metric should I use?

Accuracy measures the proportion of correctly classified instances out of the total instances. In the context of a bank loan default problem, accuracy tells you the overall proportion of correct predictions made by your model. 

A confusion matrix is a table that summarizes the performance of a classification algorithm. It compares the actual target values with the predicted values and shows the counts of true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN). In the context of the bank loan default problem:

True Positives (TP): Instances where the model correctly predicts a loan default.
True Negatives (TN): Instances where the model correctly predicts no default.
False Positives (FP): Instances where the model incorrectly predicts a default when there is none (Type I error).
False Negatives (FN): Instances where the model incorrectly predicts no default when there is one (Type II error).

A classification report provides a summary of various evaluation metrics, including precision, recall, F1-score, and support, for each class (in binary classification, typically "positive" and "negative" classes). These metrics are calculated based on the concepts of true positives, true negatives, false positives, and false negatives:

Precision: Precision measures the proportion of true positive predictions out of all positive predictions made by the model. In the context of a bank loan default problem, precision tells you how many of the predicted defaults are actual defaults. 

Recall (or Sensitivity): Recall measures the proportion of true positive predictions out of all actual positive instances. In the context of a bank loan default problem, recall tells you how many of the actual defaults were correctly predicted by the model. 

F1-score: The F1-score is the harmonic mean of precision and recall. It provides a balance between precision and recall. It is particularly useful when the classes are imbalanced.

Support: Support is the number of actual occurrences of the class in the specified dataset.

I believe looking at recall will be the best metric for this problem as we are trying to minimize false negatives (Type II errors) because we don't want the model to predict that a customer will not default (negative prediction) when they actually do default.

Recall at .69 is not very good. Let's see if I can improve the model with hyperparameter tuning.

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2']}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='recall')

grid_search.fit(X_train_resampled_controlled_df, y_train_flat)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_lr_model = grid_search.best_estimator_

In [ ]:
y_pred_best_lr = best_lr_model.predict(X_test_transformed_df)

accuracy_best_lr = accuracy_score(y_test_flat, y_pred_best_lr)
print("Accuracy:", accuracy_best_lr)

conf_matrix_best_lr = confusion_matrix(y_test_flat, y_pred_best_lr)
print("Confusion Matrix:")
print(conf_matrix_best_lr)

class_report_best_lr = classification_report(y_test_flat, y_pred_best_lr)
print("Classification Report:")
print(class_report_best_lr)

There is no change in performance from hyperparameter tuning. Next I will try Principal Component Analysis to see if reducing dimensionality will improve my model.

In [ ]:
desired_ratios = [0.6, 0.65, 0.50, 0.55]

# Load X_test_transformed_df and y_test_flat
X_test_transformed_df = pd.read_csv('X_test_transformed.csv')
y_test_flat = pd.read_csv('y_test.csv').values.ravel()

# Define hyperparameter grid
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l2']}

for desired_ratio in desired_ratios:
    # Load resampled and preprocessed data
    X_train_resampled_df = pd.read_csv(f'X_train_preprocessed_ratio_{str(int(desired_ratio * 100)).zfill(2)}_.csv')
    y_train_resampled_df = pd.read_csv(f'y_train_preprocessed_ratio_{str(int(desired_ratio * 100)).zfill(2)}_.csv')
    y_train_flat = y_train_resampled_df.values.ravel()

    # Fit logistic regression model
    lr_model = LogisticRegression()
    lr_model.fit(X_train_resampled_df, y_train_flat)

    # Predict using the fitted model
    y_pred_lr = lr_model.predict(X_test_transformed_df)

    # Evaluate the model
    accuracy = accuracy_score(y_test_flat, y_pred_lr)
    print(f"Accuracy (Desired Ratio {desired_ratio}): {accuracy}")

    conf_matrix = confusion_matrix(y_test_flat, y_pred_lr)
    print(f"Confusion Matrix (Desired Ratio {desired_ratio}):")
    print(conf_matrix)

    class_report = classification_report(y_test_flat, y_pred_lr)
    print(f"Classification Report (Desired Ratio {desired_ratio}):")
    print(class_report)

    # Hyperparameter tuning
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='recall')
    grid_search.fit(X_train_resampled_df, y_train_flat)

    # Best hyperparameters
    best_params = grid_search.best_params_
    print(f"Best Hyperparameters (Desired Ratio {desired_ratio}):", best_params)

    best_lr_model = grid_search.best_estimator_

    # Evaluate best model
    y_pred_best_lr = best_lr_model.predict(X_test_transformed_df)

    accuracy_best_lr = accuracy_score(y_test_flat, y_pred_best_lr)
    print(f"Accuracy (Best Model, Desired Ratio {desired_ratio}):", accuracy_best_lr)

    conf_matrix_best_lr = confusion_matrix(y_test_flat, y_pred_best_lr)
    print(f"Confusion Matrix (Best Model, Desired Ratio {desired_ratio}):")
    print(conf_matrix_best_lr)

    class_report_best_lr = classification_report(y_test_flat, y_pred_best_lr)
    print(f"Classification Report (Best Model, Desired Ratio {desired_ratio}):")
    print(class_report_best_lr)

Switching to markdown... not using this model but I don't want to delete the code.'

pca = PCA(n_components=0.95)

X_train_pca = pca.fit_transform(X_train_resampled_controlled_df)
X_test_pca = pca.transform(X_test_transformed_df)

lr_model_pca = LogisticRegression()
lr_model_pca.fit(X_train_pca, y_train_flat)

y_pred_lr_pca = lr_model_pca.predict(X_test_pca)

accuracy_lr_pca = accuracy_score(y_test_flat, y_pred_lr_pca)
print("Accuracy with PCA:", accuracy_lr_pca)

conf_matrix_lr_pca = confusion_matrix(y_test_flat, y_pred_lr_pca)
print("Confusion Matrix with PCA:")
print(conf_matrix_lr_pca)

class_report_lr_pca = classification_report(y_test_flat, y_pred_lr_pca)
print("Classification Report with PCA:")
print(class_report_lr_pca)

With PCA, there is still no improvement.  Next, I will look at a different ML model - Random Forest Classifier.

In [ ]:
desired_ratios = [0.6, 0.65, 0.50, 0.55]

X_test_transformed_df = pd.read_csv('X_test_transformed.csv')
y_test_flat = pd.read_csv('y_test.csv').values.ravel()

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

for desired_ratio in desired_ratios:
    X_train_resampled_df = pd.read_csv(f'X_train_preprocessed_ratio_{str(int(desired_ratio * 100)).zfill(2)}_.csv')
    y_train_resampled_df = pd.read_csv(f'y_train_preprocessed_ratio_{str(int(desired_ratio * 100)).zfill(2)}_.csv')
    y_train_flat = y_train_resampled_df.values.ravel()

    # Initialize RandomForestClassifier
    rf_model = RandomForestClassifier()

    # Fit the model
    rf_model.fit(X_train_resampled_df, y_train_flat)

    # Predict using the fitted model
    y_pred_rf = rf_model.predict(X_test_transformed_df)

    # Evaluate the model
    accuracy_rf = accuracy_score(y_test_flat, y_pred_rf)
    print(f"Accuracy (Desired Ratio {desired_ratio}) with RandomForestClassifier:", accuracy_rf)

    conf_matrix_rf = confusion_matrix(y_test_flat, y_pred_rf)
    print(f"Confusion Matrix (Desired Ratio {desired_ratio}) with RandomForestClassifier:")
    print(conf_matrix_rf)

    class_report_rf = classification_report(y_test_flat, y_pred_rf)
    print(f"Classification Report (Desired Ratio {desired_ratio}) with RandomForestClassifier:")
    print(class_report_rf)

    # Hyperparameter tuning
    grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='recall', n_jobs=-1, verbose=2)
    grid_search.fit(X_train_resampled_df, y_train_flat)

    # Best hyperparameters
    best_params = grid_search.best_params_
    print(f"Best Hyperparameters (Desired Ratio {desired_ratio}):", best_params)

    best_rf_model = grid_search.best_estimator_

    # Evaluate best model
    y_pred_best_rf = best_rf_model.predict(X_test_transformed_df)

    accuracy_best_rf = accuracy_score(y_test_flat, y_pred_best_rf)
    print(f"Accuracy (Best Model, Desired Ratio {desired_ratio}):", accuracy_best_rf)

    conf_matrix_best_rf = confusion_matrix(y_test_flat, y_pred_best_rf)
    print(f"Confusion Matrix (Best Model, Desired Ratio {desired_ratio}):")
    print(conf_matrix_best_rf)

    class_report_best_rf = classification_report(y_test_flat, y_pred_best_rf)
    print(f"Classification Report (Best Model, Desired Ratio {desired_ratio}):")
    print(class_report_best_rf)


Recall decreased significantly from .69 (lr_model) to .43. Even though accuracy improved, recall decreased significantly. For this problem, I am not overly concerned with accuracy but rather recall. 

By hyperparameter tuning, recall increased slightly but it is still significantly lower than the Logistic Regression model. Rather than spend more time on the rf_model, I am going to try a different ML model.  The next ML model I will try will be Support Vector Classifier.

NOT USING

svc_model = SVC()

svc_model.fit(X_train_resampled_controlled_df, y_train_flat)

y_pred_svc = svc_model.predict(X_test_transformed_df)

accuracy_svc = accuracy_score(y_test_flat, y_pred_svc)
print("Accuracy with SVC:", accuracy_svc)

conf_matrix_svc = confusion_matrix(y_test_flat, y_pred_svc)
print("Confusion Matrix with SVC:")
print(conf_matrix_svc)

class_report_svc = classification_report(y_test_flat, y_pred_svc)
print("Classification Report with SVC:")
print(class_report_svc)

param_grid = {
    'C': [0.1, 1, 10],                # regularization parameter
    'kernel': ['linear', 'rbf'],      # kernel type
    'gamma': ['scale', 'auto'],       # kernel coefficient (only for rbf kernel)
}

svc_model = SVC()

grid_search = GridSearchCV(svc_model, param_grid, cv=5, scoring='recall', n_jobs=-1)

grid_search.fit(X_train_resampled_controlled_df, y_train_flat)

best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

best_svc_model = grid_search.best_estimator_

y_pred_best_svc = best_svc_model.predict(X_test_transformed_df)

accuracy_best_svc = accuracy_score(y_test_flat, y_pred_best_svc)
print("Accuracy with Best SVC Model:", accuracy_best_svc)

conf_matrix_best_svc = confusion_matrix(y_test_flat, y_pred_best_svc)
print("Confusion Matrix with Best SVC Model:")
print(conf_matrix_best_svc)

class_report_best_svc = classification_report(y_test_flat, y_pred_best_svc)
print("Classification Report with Best SVC Model:")
print(class_report_best_svc)

In [ ]:
desired_ratios = [0.6, 0.65, 0.50, 0.55]

# Load X_test_transformed_df and y_test_flat
X_test_transformed_df = pd.read_csv('X_test_transformed.csv')
y_test_flat = pd.read_csv('y_test.csv').values.ravel()

# Define hyperparameter grid for XGBoost
param_grid = {
    'max_depth': [3, 6, 9],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

for desired_ratio in desired_ratios:
    X_train_resampled_df = pd.read_csv(f'X_train_preprocessed_ratio_{str(int(desired_ratio * 100)).zfill(2)}_.csv')
    y_train_resampled_df = pd.read_csv(f'y_train_preprocessed_ratio_{str(int(desired_ratio * 100)).zfill(2)}_.csv')
    y_train_flat = y_train_resampled_df.values.ravel()

    xgb_model = XGBClassifier()

    xgb_model.fit(X_train_resampled_df, y_train_flat)

    y_pred_xgb = xgb_model.predict(X_test_transformed_df)

    # Evaluate the model
    accuracy_xgb = accuracy_score(y_test_flat, y_pred_xgb)
    print(f"Accuracy (Desired Ratio {desired_ratio}) with XGBoost:", accuracy_xgb)

    conf_matrix_xgb = confusion_matrix(y_test_flat, y_pred_xgb)
    print(f"Confusion Matrix (Desired Ratio {desired_ratio}) with XGBoost:")
    print(conf_matrix_xgb)

    class_report_xgb = classification_report(y_test_flat, y_pred_xgb)
    print(f"Classification Report (Desired Ratio {desired_ratio}) with XGBoost:")
    print(class_report_xgb)

    # Hyperparameter tuning
    grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='recall', n_jobs=-1)
    grid_search.fit(X_train_resampled_df, y_train_flat)

    # Best hyperparameters
    best_params = grid_search.best_params_
    print(f"Best Hyperparameters (Desired Ratio {desired_ratio}) with XGBoost:", best_params)

    best_xgb_model = grid_search.best_estimator_

    # Evaluate best model
    y_pred_best_xgb = best_xgb_model.predict(X_test_transformed_df)

    accuracy_best_xgb = accuracy_score(y_test_flat, y_pred_best_xgb)
    print(f"Accuracy (Best Model, Desired Ratio {desired_ratio}) with XGBoost:", accuracy_best_xgb)

    conf_matrix_best_xgb = confusion_matrix(y_test_flat, y_pred_best_xgb)
    print(f"Confusion Matrix (Best Model, Desired Ratio {desired_ratio}) with XGBoost:")
    print(conf_matrix_best_xgb)

    class_report_best_xgb = classification_report(y_test_flat, y_pred_best_xgb)
    print(f"Classification Report (Best Model, Desired Ratio {desired_ratio}) with XGBoost:")
    print(class_report_best_xgb)

Recall improved to 0.69 but it is about the same as the lr_model.


# Matthew's Correlation Coefficient (MCC)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

lr_model = LogisticRegression()

lr_model.fit(X_train_resampled_df, y_train_flat)

y_pred_lr = lr_model.predict(X_test_transformed_df)

mcc_lr = matthews_corrcoef(y_test_flat, y_pred_lr)
print("Logistic Regression MCC:", mcc_lr)

print("Confusion Matrix:")
print(confusion_matrix(y_test_flat, y_pred_lr))
print("Classification Report:")
print(classification_report(y_test_flat, y_pred_lr))


In [ ]:
# Define the parameter grid for Logistic Regression
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear']
}

# Initialize GridSearchCV with MCC as the scoring metric
grid_search_lr = GridSearchCV(LogisticRegression(), param_grid_lr, cv=5, scoring='matthews_corrcoef', n_jobs=-1, verbose=2)

# Fit the model
grid_search_lr.fit(X_train_resampled_df, y_train_flat)

# Get the best parameters and model
best_params_lr = grid_search_lr.best_params_
print("Best Hyperparameters for Logistic Regression:", best_params_lr)

best_lr_model = grid_search_lr.best_estimator_

# Predict and evaluate the best model
y_pred_best_lr = best_lr_model.predict(X_test_transformed_df)
mcc_best_lr = matthews_corrcoef(y_test_flat, y_pred_best_lr)
print("Best Logistic Regression MCC:", mcc_best_lr)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test_flat, y_pred_best_lr))
print("Classification Report:")
print(classification_report(y_test_flat, y_pred_best_lr))


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV with MCC as the scoring metric
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='matthews_corrcoef', n_jobs=-1, verbose=2)

# Fit the model
grid_search_rf.fit(X_train_resampled_df, y_train_flat)

# Get the best parameters and model
best_params_rf = grid_search_rf.best_params_
print("Best Hyperparameters for Random Forest:", best_params_rf)

best_rf_model = grid_search_rf.best_estimator_

# Predict and evaluate the best model
y_pred_best_rf = best_rf_model.predict(X_test_transformed_df)
mcc_best_rf = matthews_corrcoef(y_test_flat, y_pred_best_rf)
print("Best Random Forest MCC:", mcc_best_rf)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test_flat, y_pred_best_rf))
print("Classification Report:")
print(classification_report(y_test_flat, y_pred_best_rf))


### XGBoost

In [ ]:
import xgboost as xgb

# Define the parameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Initialize GridSearchCV with MCC as the scoring metric
grid_search_xgb = GridSearchCV(xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'), param_grid_xgb, cv=5, scoring='matthews_corrcoef', n_jobs=-1, verbose=2)

# Fit the model
grid_search_xgb.fit(X_train_resampled_df, y_train_flat)

# Get the best parameters and model
best_params_xgb = grid_search_xgb.best_params_
print("Best Hyperparameters for XGBoost:", best_params_xgb)

best_xgb_model = grid_search_xgb.best_estimator_

# Predict and evaluate the best model
y_pred_best_xgb = best_xgb_model.predict(X_test_transformed_df)
mcc_best_xgb = matthews_corrcoef(y_test_flat, y_pred_best_xgb)
print("Best XGBoost MCC:", mcc_best_xgb)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test_flat, y_pred_best_xgb))
print("Classification Report:")
print(classification_report(y_test_flat, y_pred_best_xgb))
